In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
import sys
#sys.path.insert(0, '../../')
sys.path.insert(0, '/home/luka/Developement/accetuation/')
from prepare_data import *

%run ../../prepare_data.py

In [3]:
data = Data('s')
data.generate_data('syllables_word_accetuation_train',
                   'syllables_word_accetuation_test',
                   'syllables_word_accetuation_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [4]:
content = data._read_content('../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [5]:
pickle_input = open('letters_word_accetuation_predictions.pkl', 'rb')
letters_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_word_accetuation_predictions.pkl', 'rb')
syllables_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_word_accetuation_predictions.pkl', 'rb')
syllabled_letters_word_accetuation_predictions = pickle.load(pickle_input)

In [10]:
ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions ]), axis=0 )
accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)

91.7054055092397
